#### Importing Dependencies

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import os
import numpy as np
from shutil import copyfile
from nibabel.testing import data_path
import nibabel as nib
import matplotlib.pyplot as plt
from nilearn.plotting import view_img, glass_brain, plot_anat, plot_epi
import glob
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

/usr/local/lib/python3.5/dist-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


#### Importing Preprocessed CSV files

In [2]:
data=pd.read_csv('data.csv')
data

,filename,target
0,3908_7_functional.nii,0
1,3418_functional.nii,1
2,4104_4_functional.nii,0
3,4104_6_functional.nii,0
4,4100_4_functional.nii,0
...,...,...
119,3917_4_functional.nii,0
120,3460_4_functional.nii,0
121,3444_functional.nii,1
122,3237_2_functional.nii,0


#### Importing Data

In [3]:
ff = glob.glob('data/all_data/*')

In [4]:
len(ff)

124

#### Initializing an array to store images

In [5]:
images = []
print(images)

[]


#### Loading inages
#### Shape Correction (taking middle 30 slides for each Scan)
#### Appending the images into Array

In [6]:
for f in range(len(ff)):
    a = nib.load(ff[f])
    print(ff[f])
    l1,l2,l3=a.shape
    #print(l1,l2,l3)
    a = a.get_data()
    #print(a)
    
    z1= int(l3/2) -15
    z2= int(l3/2) +15
    a = a[:,:,z1:z2]
    #print(a)
    #for i in range (len(ff)):
    images.append((a[:,:,:]))
    #print (a.shape)
#print(images)

data/all_data/3908_7_functional.nii
data/all_data/3418_functional.nii
data/all_data/4104_4_functional.nii
data/all_data/4104_6_functional.nii
data/all_data/4100_4_functional.nii
data/all_data/4055_functional.nii
data/all_data/3666_functional.nii
data/all_data/3237_3_functional.nii
data/all_data/3551_2_functional.nii
data/all_data/4100_2_functional.nii
data/all_data/3201_3_functional.nii
data/all_data/3433_functional.nii
data/all_data/3431_functional.nii
data/all_data/3448_functional.nii
data/all_data/4102_functional.nii
data/all_data/3443_functional.nii
data/all_data/4104_2_functional.nii
data/all_data/4111_functional.nii
data/all_data/3225_functional.nii
data/all_data/3551_3_functional.nii
data/all_data/4094_functional.nii
data/all_data/4116_5_functional.nii
data/all_data/3901_3_functional.nii
data/all_data/3917_1_functional.nii
data/all_data/4116_2_functional.nii
data/all_data/4117_functional.nii
data/all_data/3132_functional.nii
data/all_data/3908_2_functional.nii
data/all_data/3235

In [7]:
imagess=np.asarray(images)

In [8]:
imagess.shape

(124, 128, 128, 30)

#### Min Max Normalization

In [9]:
m = np.max(imagess)
mi = np.min(imagess)

In [10]:
print(m, mi)

32767.0 -607.0


In [11]:
images = (images - mi) / (m - mi)

In [12]:
np.min(images), np.max(images)

(0.0, 1.0)

In [13]:
imagess

array([[[[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        ...,

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
    

# CNN

#### Importing depedencies for CNN

In [14]:
from keras.layers import Conv2D, MaxPooling2D, Conv3D, MaxPool3D, Flatten, Dense
from keras.layers import Dropout, Input, BatchNormalization
from sklearn.metrics import confusion_matrix, accuracy_score
from plotly.offline import iplot, init_notebook_mode
from keras.losses import categorical_crossentropy
from keras.optimizers import Adadelta
import plotly.graph_objs as go
from matplotlib.pyplot import cm
from keras.models import Model
import numpy as np
import keras
from keras import backend as K
from keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical
from keras.models import Sequential, model_from_json,Model
from keras.optimizers import SGD, RMSprop, Adam, Adagrad, Adadelta
import sklearn
from sklearn import model_selection
from sklearn.model_selection import train_test_split, KFold, cross_val_score, StratifiedKFold, learning_curve, GridSearchCV
from sklearn.metrics import confusion_matrix, make_scorer, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
import itertools
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization, Conv2D, MaxPool2D, MaxPooling2D, Input, Convolution2D
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold

Using TensorFlow backend.


#### Defining Function for
##### Saving MetricsCheckpoint
##### Plotting Confusion Matrix
##### Plotting Learning Curve

In [17]:
# Helper Functions  Learning Curves and Confusion Matrix

class MetricsCheckpoint(Callback):
    """Callback that saves metrics after each epoch"""
    def __init__(self, savepath):
        super(MetricsCheckpoint, self).__init__()
        self.savepath = savepath
        self.history = {}
    def on_epoch_end(self, epoch, logs=None):
        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)
        np.save(self.savepath, self.history)
def plotKerasLearningCurve():
    plt.figure(figsize=(10,5))
    metrics = np.load('logs.npy')[()]
    filt = ['acc'] # try to add 'loss' to see the loss learning curve
    for k in filter(lambda x : np.any([kk in x for kk in filt]), metrics.keys()):
        l = np.array(metrics[k])
        plt.plot(l, c= 'r' if 'val' not in k else 'b', label='val' if 'val' in k else 'train')
        x = np.argmin(l) if 'loss' in k else np.argmax(l)
        y = l[x]
        plt.scatter(x,y, lw=0, alpha=0.25, s=100, c='r' if 'val' not in k else 'b')
        plt.text(x, y, '{} = {:.4f}'.format(x,y), size='15', color= 'r' if 'val' not in k else 'b')   
    plt.legend(loc=4)
    plt.axis([0, None, None, None]);
    plt.grid()
    plt.xlabel('Number of epochs')

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.figure(figsize = (5,5))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=90)
    plt.yticks(tick_marks, classes)
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

def plot_learning_curve(history):
    plt.figure(figsize=(8,8))
    plt.subplot(1,2,1)
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.savefig('./accuracy_curve.png')
    #plt.clf()
    # summarize history for loss
    plt.subplot(1,2,2)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.savefig('./loss_curve.png')

In [18]:
imagess.shape

(124, 128, 128, 30)

#### Reading output Labels
##### PD=1, Control=0

In [19]:
data=pd.read_csv('data.csv')
y=data.target
print(y)

0      0
1      1
2      0
3      0
4      0
      ..
119    0
120    0
121    1
122    0
123    0
Name: target, Length: 124, dtype: int64


#### K=4 fold

In [20]:
seed = 13
np.random.seed(13)
cvscores = []
y1= keras.utils.to_categorical(y, 2)

 n_split = 4
count=1
for train, test in KFold(n_split).split(imagess):
    x_traincnn, x_testcnn=imagess[train],imagess[test]
    y_traincnn, y_testcnn=y1[train],y1[test]
    ## input layer
    input_layer = Input((128, 128, 30))

    ## convolutional layers
    conv_layer1 = Conv2D(filters=32, kernel_size=(3, 3), activation='relu',padding='same')(input_layer)
    conv_layer2 = Conv2D(filters=32, kernel_size=(3, 3), activation='relu',padding='same')(conv_layer1)

    ## add max pooling to obtain the most imformatic features
    pooling_layer1 = MaxPooling2D(pool_size=(2, 2))(conv_layer2)

    conv_layer3 = Conv2D(filters=32, kernel_size=(3, 3), activation='relu')(pooling_layer1)
    conv_layer4 = Conv2D(filters=32, kernel_size=(3, 3), activation='relu')(conv_layer3)
    pooling_layer2 = MaxPooling2D(pool_size=(2, 2))(conv_layer4)

    ## perform batch normalization on the convolution outputs before feeding it to MLP architecture
    pooling_layer2 = BatchNormalization()(pooling_layer2)
    flatten_layer = Flatten()(pooling_layer2)

    ## create an MLP architecture with dense layers : 2048 -> 512 -> 2
    ## add dropouts to avoid overfitting / perform regularization
    dense_layer1 = Dense(units=2048, activation='relu')(flatten_layer)
    dense_layer1 = Dropout(0.5)(dense_layer1)
    dense_layer2 = Dense(units=512, activation='relu')(dense_layer1)
    dense_layer2 = Dropout(0.5)(dense_layer2)
    output_layer = Dense(units=2, activation='softmax')(dense_layer2)

    ## define the model with input layer and output layer
    model = Model(inputs=input_layer, outputs=output_layer)
    learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)
    checkpointer = ModelCheckpoint('pd_cnn.h5',monitor='val_acc', verbose=1, save_best_only=True)
    earlystopper = EarlyStopping(patience=5, verbose=1)
    model.compile(loss=categorical_crossentropy, optimizer=Adadelta(lr=0.1), metrics=['acc'])
    # Without data augmentation
    history = model.fit(x_traincnn, y_traincnn, batch_size = 20, epochs = 15, validation_data = (x_testcnn, y_testcnn), verbose = 2, callbacks = [MetricsCheckpoint('logs'),earlystopper,learning_rate_reduction,checkpointer])
    score = model.evaluate(x_testcnn, y_testcnn, verbose=0)
    if count==1:
        accu1=score[1]
    elif count==2:
        accu2=score[1]
    elif count==3:
        accu3=score[1]
    elif count==4:
        accu4=score[1]
    #elif count==5:
        #accu5=score[1]
    print('.................................................\n\n\n')
    print('\nKeras CNN  #', count,' - accuracy:', score[1],'\n')
    print('.................................................\n\n\n')
    count=count+1
    y_pred = model.predict(x_testcnn)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Train on 93 samples, validate on 31 samples
Epoch 1/15
 - 10s - loss: 3.7280 - acc: 0.5269 - val_loss: 12.2680 - val_acc: 0.5806

Epoch 00001: val_acc improved from -inf to 0.58065, saving model to pd_cnn.h5
Epoch 2/15
 - 8s - loss: 3.2037 - acc: 0.6022 - val_loss: 1.8700 - val_acc: 0.4839

Epoch 00002: val_acc did not improve from 0.58065
Epoch 3/15
 - 8s - loss: 1.7424 - acc: 0.6989 - val_loss: 2.8250 - val_acc: 0.4839

Epoch 00003: val_acc did not improve from 0.58065
Epoch 4/15
 - 8s - loss: 1.3966 - acc: 0.6559 - val_loss: 1.1399 - val_acc: 0.7419

Epoch 00004: val_acc improved from 0.58065 to 0.74194, saving model to pd_cnn.h5
Epoch 5/15
 - 8s - loss: 1.2378 - acc: 0.6667 - val_loss: 1.9149 - val_acc: 0.4839

Epoch 00005: val_acc did not improve from 0.74194
Epoch 6/15
 - 8s - loss: 1.0470 - acc: 0.6989 - val_loss: 1.4056 - val_acc: 0.5161

Epoch 00006: val_acc did not improve from 0.74194
Epoch 7/

In [22]:
print(accu1)
print(accu2)
print(accu3)
print(accu4)
#print(accu5)
print((accu1+accu2+accu3+accu4)/4)


0.8709677457809448
0.8709677457809448
0.8064516186714172
0.8709677457809448
0.8548387140035629
